# ECSN Test

In [1]:
from StarKiller.initialization import starkiller_initialize
from StarKiller.interfaces import EosType
from StarKiller.interfaces import BurnType
from StarKiller.eos import Eos
from StarKiller.network import Network
import numpy as np

In [2]:
probin_file = "probin_ecsn"

In [3]:
starkiller_initialize(probin_file)

In [4]:
helmholtz = Eos()
ecsn = Network()

## Test 1 from Josiah

In [5]:
burn_state_1 = BurnType()

burn_state_1.state.rho = 7607783959.50421e0
burn_state_1.state.t = 983559309.749412e0
burn_state_1.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["he4"]] = 1.14998095787427E-18
burn_state_1.state.xn[ecsn.species_map["o16"]] = 0.5955320774
burn_state_1.state.xn[ecsn.species_map["o20"]] = 0.203902681
burn_state_1.state.xn[ecsn.species_map["f20"]] = 1.17278643621995E-07
burn_state_1.state.xn[ecsn.species_map["ne20"]] = 0.19753136
burn_state_1.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["si28"]] = 0.0027259815
burn_state_1.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["s32"]] = 0.0

In [6]:
ecsn.rhs(burn_state_1)

In [7]:
print(burn_state_1.state)

<burn_t>{
    rho : 7607783959.50421,
    t : 983559309.749412,
    e : 2.490743133790312e+18,
    xn : array([0.00000000e+00, 1.14998096e-18, 5.95532077e-01, 2.03902681e-01,
       1.17278644e-07, 1.97531360e-01, 0.00000000e+00, 0.00000000e+00,
       2.72598150e-03, 0.00000000e+00, 0.00000000e+00]),
    cv : 14444247.553061701,
    cp : 14486626.878325198,
    y_e : 0.47945583462538965,
    eta : 87.75746182063078,
    cs : 1099451413.7881703,
    dx : 0.0,
    abar : 17.42469332287015,
    zbar : 8.354370880208164,
    t_old : 0.0,
    dcvdt : 0.0,
    dcpdt : 3.162e-320,
    ydot : array([ 1.16695205e-11, -7.26850364e-12, -4.11345965e-11,  5.79382894e-11,
        7.42255606e-12, -5.81959032e-11,  2.16734518e-12,  0.00000000e+00,
        4.23112889e-12,  1.16695205e-11,  5.05156388e-16, -3.95207429e+01,
       -5.72522257e+08]),
    jac : array([[0.00000000e+000, 3.16202013e-320, 0.00000000e+000,
        0.00000000e+000, 4.24400499e-314, 0.00000000e+000,
        0.00000000e+000, 4.0

In [8]:
Hnuc_1 = burn_state_1.state.ydot[ecsn.net_ienuc]
print("Energy generation rate: {}".format(Hnuc_1))

Energy generation rate: -572522256.6568681


In [22]:
eos_state_1 = burn_state_1.to_eos_type()
helmholtz.evaluate(eos_state_1.eos_input_rt, eos_state_1)

In [10]:
print(eos_state_1.state)

<eos_t>{
    rho : 7607783959.50421,
    t : 983559309.749412,
    p : 6.87649058595149e+27,
    e : 2.490743133790312e+18,
    h : 3.3946187725511716e+18,
    s : 46493486.57937405,
    xn : array([0.00000000e+00, 1.14998096e-18, 5.95532077e-01, 2.03902681e-01,
       1.17278644e-07, 1.97531360e-01, 0.00000000e+00, 0.00000000e+00,
       2.72598150e-03, 0.00000000e+00, 0.00000000e+00]),
    aux : array([], dtype=float64),
    dpdt : 5.482451265458468e+16,
    dpdr : 1.2052571947699988e+18,
    dedt : 14444247.553061701,
    dedr : 117877656.67151032,
    dhdt : 21650618.41564302,
    dhdr : 157492550.78698152,
    dsdt : 0.014685690440713491,
    dsdr : -0.0009472365278694044,
    dpde : 3795594921.3127203,
    dpdr_e : 7.578413597713697e+17,
    cv : 14444247.553061701,
    cp : 14486626.878325198,
    xne : 2.196634277233492e+33,
    xnp : 0.0,
    eta : 87.75746182063078,
    pele : 6.958956124711135e+27,
    ppos : 0.0,
    mu : 0.0,
    mu_e : 2.085697842808241,
    y_e : 0.47945

In [11]:
omegadot_1 = burn_state_1.state.xn[:ecsn.nspec_evolve]
cp_1 = eos_state_1.state.cp
dpdt_1 = eos_state_1.state.dpdt
dpdx_1 = eos_state_1.state.dpdx
dhdx_1 = eos_state_1.state.dhdx

dHcompdx_1 = (cp_1 * dpdx_1/dpdt_1 - dhdx_1)
Hcomp_1 = np.dot(dHcompdx_1, omegadot_1)

print("Composition energy generation: {}".format(Hcomp_1))

Composition energy generation: -299087087649827.1


In [12]:
Htot_1 = Hnuc_1 + Hcomp_1
print("Total energy generation rate: {}".format(Htot_1))

Total energy generation rate: -299087660172083.8


## Test 2 from Josiah

In [13]:
burn_state_2 = BurnType()

burn_state_2.state.rho = 8346403698.95185e0
burn_state_2.state.t = 656536305.140964e0
burn_state_2.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["he4"]] = 8.89957317703129E-23
burn_state_2.state.xn[ecsn.species_map["o16"]] = 0.5999942706e0
burn_state_2.state.xn[ecsn.species_map["o20"]] = 0.3926652894e0
burn_state_2.state.xn[ecsn.species_map["f20"]] = 0.000000002e0
burn_state_2.state.xn[ecsn.species_map["ne20"]] = 0.0073364571e0
burn_state_2.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["si28"]] = 3.518236231242E-06
burn_state_2.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["s32"]] = 0.0

In [14]:
ecsn.rhs(burn_state_2)

In [15]:
print(burn_state_2.state)

<burn_t>{
    rho : 8346403698.95185,
    t : 656536305.140964,
    e : 2.4366503017450455e+18,
    xn : array([0.00000000e+00, 8.89957318e-23, 5.99994271e-01, 3.92665289e-01,
       2.00000000e-09, 7.33645710e-03, 0.00000000e+00, 0.00000000e+00,
       3.51823623e-06, 0.00000000e+00, 0.00000000e+00]),
    cv : 13444158.983137527,
    cp : 13470518.44019436,
    y_e : 0.46073323962811563,
    eta : 133.98527882572262,
    cs : 1086123519.6557314,
    dx : 0.0,
    abar : 17.391348207658748,
    zbar : 8.012772201215236,
    t_old : 0.0,
    dcvdt : 2e-323,
    dcpdt : 2.1220085655e-314,
    ydot : array([ 3.75545872e-17, -2.13349498e-17, -1.35886276e-16,  1.24835159e-12,
        1.35222972e-13, -1.38354101e-12,  5.60711361e-19,  0.00000000e+00,
        1.33342876e-17,  3.75545872e-17,  4.90756421e-25, -1.19809216e+00,
       -1.61389226e+07]),
    jac : array([[0.00000000e+000, 6.36600002e-314, 0.00000000e+000,
        0.00000000e+000, 2.12200857e-314, 0.00000000e+000,
        3.952525

In [16]:
Hnuc_2 = burn_state_2.state.ydot[ecsn.net_ienuc]
print("Energy generation rate: {}".format(Hnuc_2))

Energy generation rate: -16138922.581570987


In [17]:
eos_state_2 = burn_state_2.to_eos_type()
helmholtz.evaluate(eos_state_2.eos_input_rt, eos_state_2)

In [18]:
print(eos_state_2.state)

<eos_t>{
    rho : 8346403698.95185,
    t : 656536305.140964,
    p : 7.364917477765657e+27,
    e : 2.4366503017450455e+18,
    h : 3.3190563946459377e+18,
    s : 40474438.71911317,
    xn : array([0.00000000e+00, 8.89957318e-23, 5.99994271e-01, 3.92665289e-01,
       2.00000000e-09, 7.33645710e-03, 0.00000000e+00, 0.00000000e+00,
       3.51823623e-06, 0.00000000e+00, 0.00000000e+00]),
    aux : array([], dtype=float64),
    dpdt : 5.73841729147549e+16,
    dpdr : 1.1773559024964984e+18,
    dedt : 13444158.983137527,
    dedr : 105182091.4988295,
    dhdt : 20319476.183761463,
    dhdr : 140520642.1170612,
    dsdt : 0.02047740372903058,
    dsdr : -0.0008237460646059265,
    dpde : 4268334894.4868603,
    dpdr_e : 7.284035110769347e+17,
    cv : 13444158.983137527,
    cp : 13470518.44019436,
    xne : 2.315793919641922e+33,
    xnp : 0.0,
    eta : 133.98527882572262,
    pele : 7.464603358528272e+27,
    ppos : 0.0,
    mu : 6.9482865035104e-310,
    mu_e : 2.1704533426048394,


In [19]:
omegadot_2 = burn_state_2.state.xn[:ecsn.nspec_evolve]
cp_2 = eos_state_2.state.cp
dpdt_2 = eos_state_2.state.dpdt
dpdx_2 = eos_state_2.state.dpdx
dhdx_2 = eos_state_2.state.dhdx

dHcompdx_2 = (cp_2 * dpdx_2/dpdt_2 - dhdx_2)
Hcomp_2 = np.dot(dHcompdx_2, omegadot_2)

print("Composition energy generation: {}".format(Hcomp_2))

Composition energy generation: -468360422910.3395


In [20]:
Htot_2 = Hnuc_2 + Hcomp_2
print("Total energy generation rate: {}".format(Htot_2))

Total energy generation rate: -468376561832.921
